In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

data_path = r'./archive/dataset_blood_group'

blood_groups = ['O-', 'O+', 'B-', 'B+', 'AB-', 'AB+', 'A-', 'A+']
X, y = [], []

for i, group in enumerate(blood_groups):
    group_path = os.path.join(data_path, group)
    for img_name in os.listdir(group_path):
        img_path = os.path.join(group_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img_resized = cv2.resize(img, (128, 128)) / 255.0
            X.append(img_resized)
            y.append(i)

X = np.array(X)
y = to_categorical(np.array(y), num_classes=len(blood_groups))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)


model = Sequential([
    Conv2D(32, (3, 3), activation='swish', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='swish'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='swish'),
    Dropout(0.1),
    Dense(len(blood_groups), activation='softmax')
])


model.compile(optimizer=Adam(), loss='categorical_crossentropy', 
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=20, validation_split=0.1, batch_size=32)


model.save('./blood_group_fingerprint_model.h5')


y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)
print("Test Accuracy:", accuracy_score(y_test_labels, y_pred))
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred, target_names=blood_groups))


K:\anaconda_py\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 57s 393ms/step - accuracy: 0.3235 - loss: 2.3599 - val_accuracy: 0.8108 - val_loss: 0.6373
Epoch 2/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 50s 366ms/step - accuracy: 0.7445 - loss: 0.6624 - val_accuracy: 0.8358 - val_loss: 0.4326
Epoch 3/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 50s 367ms/step - accuracy: 0.8239 - loss: 0.4847 - val_accuracy: 0.8586 - val_loss: 0.3683
Epoch 4/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 50s 370ms/step - accuracy: 0.8606 - loss: 0.3583 - val_accuracy: 0.8524 - val_loss: 0.3969
Epoch 5/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 50s 367ms/step - accuracy: 0.9067 - loss: 0.2653 - val_accuracy: 0.8836 - val_loss: 0.3194
Epoch 6/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 49s 363ms/step - accuracy: 0.9276 - loss: 0.2066 - val_accuracy: 0.9127 - val_loss: 0.2580
Epoch 7/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 49s 357ms/step - accuracy: 0.9591 - loss: 0.1337 - val_accuracy: 0.8836 - val_loss: 0.2879
Epoch 8/20
136/136 ━━━━━━━━━━━━━━━━━━━━ 49s 357ms/step - accuracy: 0.9709 - loss: 0

38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step
Test Accuracy: 0.8426311407160699

Classification Report:
               precision    recall  f1-score   support

          O-       0.88      0.73      0.80       163
          O+       0.80      0.89      0.84       184
          B-       0.94      0.83      0.88       140
          B+       0.86      0.86      0.86       124
         AB-       0.81      0.84      0.82       122
         AB+       0.80      0.86      0.83       147
          A-       0.81      0.86      0.84       197
          A+       0.91      0.87      0.89       124

    accuracy                           0.84      1201
   macro avg       0.85      0.84      0.84      1201
weighted avg       0.85      0.84      0.84      1201

